In [1]:
!nvidia-smi

Mon Apr 17 02:23:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;REFERENCES</a></h3>

Processed Data: [GISLR Feature Data: On the Shoulders](https://www.kaggle.com/code/roberthatch/gislr-feature-data-on-the-shoulders)<br>
FeatureGEN class: [GISLR Feature Data: On the Shoulders](https://www.kaggle.com/code/roberthatch/gislr-feature-data-on-the-shoulders)<br>
Chapter html format: [🤟 GISLR 🤟 - 📚Learn – 🔭EDA – 🤖Baseline
](https://www.kaggle.com/code/dschettler8845/gislr-learn-eda-baseline)<br>


<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">1&nbsp;&nbsp;&nbsp;&nbsp;INSTALL LIBRARY</a></h3>

In [2]:
!pip install tflite_runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.8 MB/s eta 0:00:00a 0:00:01


<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">2&nbsp;&nbsp;&nbsp;&nbsp;IMPORT LIBRARY</a></h3>

In [3]:
!pip install onnxsim
!pip install onnx_tf
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob

import onnx
import onnxsim
from onnx_tf.backend import prepare
import tensorflow as tf

import torch
import torch.nn as nn
import torch.optim.lr_scheduler as scheduler
from torch.utils.data import Dataset, DataLoader, random_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 8.1 MB/s eta 0:00:00


Installing onnxruntime by `/opt/conda/bin/python3.7 -m pip install --user onnxruntime`, please wait for a moment..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.5 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">3&nbsp;&nbsp;&nbsp;&nbsp;DATASET</a></h3>

In [4]:
path = '/kaggle/input/gislr-feature-data-on-the-shoulders'

In [5]:
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [6]:
class PreprocessedGISLDataset(Dataset):
    def __init__(self, path) -> None:
        super().__init__()
        
        self.dataset = np.load(os.path.join(path, "feature_data.npy"))
        self.labels = np.load(os.path.join(path, "feature_labels.npy"))
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx, :], self.labels[idx]


In [7]:
dataset = PreprocessedGISLDataset(path)

dataset_size = len(dataset)
train_size = int(dataset_size * 0.9)
validation_size = dataset_size - train_size

train_dataset, validation_dataset = random_split(dataset, [train_size, validation_size])

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)
validation_dataset = DataLoader(validation_dataset, batch_size=64, shuffle=True, drop_last=True)

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">4&nbsp;&nbsp;&nbsp;&nbsp;MODEL</a></h3>

In [8]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.in_feature = 5796
        self._1_feature = 1024
        self._2_feature = 512
        self.out_feature = 250

        self.l1 = nn.Sequential(nn.Linear(self.in_feature, self._1_feature),
                               nn.BatchNorm1d(self._1_feature),
                               nn.ReLU(),
                                nn.Dropout(p=0.4),
                               )
        
        self.l2 = nn.Sequential(nn.Linear(self._1_feature, self._2_feature),
                               nn.BatchNorm1d(self._2_feature),
                               nn.ReLU(),
                                nn.Dropout(p=0.4),
                               )
        
        self.l3 = nn.Sequential(nn.Linear(self._2_feature, self.out_feature),
                               nn.BatchNorm1d(self.out_feature),
                               nn.ReLU(),
                                nn.Dropout(p=0.4),
                               )
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input):
        out = self.l1(input)
        out = self.l2(out)
        out = self.l3(out)
        out = self.softmax(out)
        return out

In [9]:
model = SimpleNet().to("cuda")

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">5&nbsp;&nbsp;&nbsp;&nbsp;TRAINING</a></h3>

In [10]:
epochs = 250
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-6)
cosine_scheduler = scheduler.CosineAnnealingLR(optimizer=optimizer,
                            T_max=100,
                            last_epoch=-1,
                            verbose=False)

for e in range(epochs):
    train_loss = 0
    train_acc = 0
    num_iter = 0
    model.train()
    for x, y in tqdm(train_dataloader):
        num_iter+=1
        pred = model(x.to(torch.float32).to("cuda"))
        loss = loss_fn(pred.to(torch.float32), y.type(torch.LongTensor).to("cuda"))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = pred.max(1).indices
        train_acc+=(torch.sum(pred == y.to("cuda"))/x.shape[0])
    cosine_scheduler.step()

    print(f"Training Acc @ epoch {e}: {train_acc/num_iter*100}%")

    val_loss = 0
    val_acc = 0
    num_iter = 0
    model.eval()
    for x, y in validation_dataset:
        num_iter+=1
        pred = model(x.to("cuda").to(torch.float32))
        loss = loss_fn(pred.to(torch.float32), y.type(torch.LongTensor).to("cuda"))
        
        pred = pred.max(1).indices
        val_acc+=(torch.sum(pred == y.to("cuda"))/x.shape[0])
    print(f"Validation Acc @ epoch {e}: {val_acc/num_iter*100}%")

test_loss = 0
test_acc = 0
num_iter = 0
model.eval()
# for x, y in validation_dataset:
#     num_iter+=1
#     pred = model(x.to("cuda").to(torch.float32))
#     loss = loss_fn(pred.to(torch.float32), y.type(torch.LongTensor).to("cuda"))

#     pred = pred.max(1).indices
#     test_acc+=(torch.sum(pred == y.to("cuda"))/x.shape[0])
# print(f"Test Acc: {test_acc/num_iter}%")


100%|██████████| 332/332 [00:05<00:00, 66.18it/s] 


Training Acc @ epoch 0: 1.0942206382751465%
Validation Acc @ epoch 0: 2.7104592323303223%


100%|██████████| 332/332 [00:03<00:00, 99.79it/s] 


Training Acc @ epoch 1: 2.723785638809204%
Validation Acc @ epoch 1: 4.719387531280518%


100%|██████████| 332/332 [00:03<00:00, 104.76it/s]


...

100%|██████████| 332/332 [00:03<00:00, 105.87it/s]


Training Acc @ epoch 248: 42.63460159301758%
Validation Acc @ epoch 248: 68.26104736328125%


100%|██████████| 332/332 [00:03<00:00, 97.52it/s] 


Training Acc @ epoch 249: 43.04287338256836%
Validation Acc @ epoch 249: 67.38945007324219%


SimpleNet(
  (l1): Sequential(
    (0): Linear(in_features=5796, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (l2): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (l3): Sequential(
    (0): Linear(in_features=512, out_features=250, bias=True)
    (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (softmax): Softmax(dim=1)
)

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">6&nbsp;&nbsp;&nbsp;&nbsp;FEATURE PROCESSING MODULE</a></h3>

In [11]:
DROP_Z = False

NUM_FRAMES = 15
SEGMENTS = 3

LEFT_HAND_OFFSET = 468
POSE_OFFSET = LEFT_HAND_OFFSET + 21
RIGHT_HAND_OFFSET = POSE_OFFSET + 33

## average over the entire face, and the entire 'pose'
averaging_sets = [[0, 468], [POSE_OFFSET, 33]]

lip_landmarks = [
    61,
    185,
    40,
    39,
    37,
    0,
    267,
    269,
    270,
    409,
    291,
    146,
    91,
    181,
    84,
    17,
    314,
    405,
    321,
    375,
    78,
    191,
    80,
    81,
    82,
    13,
    312,
    311,
    310,
    415,
    95,
    88,
    178,
    87,
    14,
    317,
    402,
    318,
    324,
    308,
]
left_hand_landmarks = list(range(LEFT_HAND_OFFSET, LEFT_HAND_OFFSET + 21))
right_hand_landmarks = list(range(RIGHT_HAND_OFFSET, RIGHT_HAND_OFFSET + 21))

point_landmarks = [
    item
    for sublist in [lip_landmarks, left_hand_landmarks, right_hand_landmarks]
    for item in sublist
]

LANDMARKS = len(point_landmarks) + len(averaging_sets)
print(LANDMARKS)
if DROP_Z:
    INPUT_SHAPE = (NUM_FRAMES, LANDMARKS * 2)
else:
    INPUT_SHAPE = (NUM_FRAMES, LANDMARKS * 3)

FLAT_INPUT_SHAPE = (INPUT_SHAPE[0] + 2 * (SEGMENTS + 1)) * INPUT_SHAPE[1]

84


In [12]:
def tf_nan_mean(x, axis=0):
    return tf.reduce_sum(
        tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis
    ) / tf.reduce_sum(
        tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis
    )


def tf_nan_std(x, axis=0):
    d = x - tf_nan_mean(x, axis=axis)
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis))


def flatten_means_and_stds(x, axis=0):
    # Get means and stds
    x_mean = tf_nan_mean(x, axis=0)
    x_std = tf_nan_std(x, axis=0)

    x_out = tf.concat([x_mean, x_std], axis=0)
    x_out = tf.reshape(x_out, (1, INPUT_SHAPE[1] * 2))
    x_out = tf.where(tf.math.is_finite(x_out), x_out, tf.zeros_like(x_out))
    return x_out

In [13]:
class FeatureGen(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureGen, self).__init__()
    
    def call(self, x_in):
#         print(right_hand_percentage(x))
        x_list = [tf.expand_dims(tf_nan_mean(x_in[:, av_set[0]:av_set[0]+av_set[1], :], axis=1), axis=1) for av_set in averaging_sets]
        x_list.append(tf.gather(x_in, point_landmarks, axis=1))
        x = tf.concat(x_list, 1)

        x_padded = x
        for i in range(SEGMENTS):
            p0 = tf.where( ((tf.shape(x_padded)[0] % SEGMENTS) > 0) & ((i % 2) != 0) , 1, 0)
            p1 = tf.where( ((tf.shape(x_padded)[0] % SEGMENTS) > 0) & ((i % 2) == 0) , 1, 0)
            paddings = [[p0, p1], [0, 0], [0, 0]]
            x_padded = tf.pad(x_padded, paddings, mode="SYMMETRIC")
        x_list = tf.split(x_padded, SEGMENTS)
        x_list = [flatten_means_and_stds(_x, axis=0) for _x in x_list]

        x_list.append(flatten_means_and_stds(x, axis=0))
        
        ## Resize only dimension 0. Resize can't handle nan, so replace nan with that dimension's avg value to reduce impact.
        x = tf.image.resize(tf.where(tf.math.is_finite(x), x, tf_nan_mean(x, axis=0)), [NUM_FRAMES, LANDMARKS])
        x = tf.reshape(x, (1, INPUT_SHAPE[0]*INPUT_SHAPE[1]))
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x_list.append(x)
        x = tf.concat(x_list, axis=1)
        return x


<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">7&nbsp;&nbsp;&nbsp;&nbsp;TORCH TO ONNX</a></h3>

In [14]:
x = torch.randn(2, 5796, requires_grad=True).to("cuda")
# torch_out = torch_model(x)

model = model.to("cuda")

torch.onnx.export(model,               
                  x,                   
                  "MLP.onnx",   
                  export_params=True,  
                  opset_version=10,    
                  do_constant_folding=True, 
                  input_names = ['input'],  
                  output_names = ['output'],
                  dynamic_axes={'input' : {0 : 'batch_size'},   
                                'output' : {0 : 'batch_size'}})

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">8&nbsp;&nbsp;&nbsp;&nbsp;ONNX to TF</a></h3>

In [15]:
tf_model_path = "tf_MLP"
onnx_asl_module = onnx.load("MLP.onnx")
tf_rep = prepare(onnx_asl_module)
tf_rep.export_graph(tf_model_path)

In [16]:
class TFModel(tf.Module):
    def __init__(self, tf_model_path):
        super().__init__()

        self.feature_gen = FeatureGen()
        self.model = tf.saved_model.load(tf_model_path)
        self.feature_gen.trainable = False
        self.model.trainable = False

    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name="inputs")
        ]
    )
    def call(self, input):
        output_tensors = {}
        features = self.feature_gen(tf.cast(input, dtype=tf.float32))

        output_tensors["outputs"] = self.model(**{"input": features})["output"][0, :]

        return output_tensors


mytfmodel = TFModel("./tf_MLP")
tf.saved_model.save(
    mytfmodel, "tf_infer_model", signatures={"serving_default": mytfmodel.call}
)

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: black; background-color: #ffffff;"><a style="text-decoration: none; color: black;" href="#imports">9&nbsp;&nbsp;&nbsp;&nbsp;TF to TFlite</a></h3>

In [17]:
tf_infer_model_path = "./tf_infer_model"
converter = tf.lite.TFLiteConverter.from_saved_model(tf_infer_model_path)
tflite_model = converter.convert()

tflite_model_path = "model.tflite"

# Save the model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

In [18]:
pq_path = "/kaggle/input/asl-signs/train_landmark_files/53618/1001379621.parquet"

import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter(tflite_model_path)
interpreter.allocate_tensors()

found_signatures = list(interpreter.get_signature_list().keys())

# if REQUIRED_SIGNATURE not in found_signatures:
#     raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=load_relevant_data_subset(pq_path))
sign = np.argmax(output["outputs"])

print(sign, output["outputs"].shape)

30 (250,)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [19]:
!zip submission.zip $tflite_model_path

  adding: model.tflite (deflated 7%)
